# 🧬 Generación de Datos Sintéticos con SDV

**Versión:** v3_experimental  
**Rol:** Data Engineer (Synthetic Data)  
**Fecha:** 2026-01-08

---

## Objetivo

Generar datos sintéticos tabulares usando **GaussianCopulaSynthesizer** de SDV para aumentar el dataset de entrenamiento.

## Método

**GaussianCopula** es el método recomendado para datasets pequeños (<1000 filas) porque:
- No requiere muchos datos para aprender distribuciones
- Captura correlaciones entre variables
- Es estable y reproducible

## Restricciones

> ⚠️ **CERO LEAKAGE**: Solo se usa el archivo de Train. El Test está sellado.

---

In [1]:
# ==============================================================================
# CONFIGURACIÓN Y DEPENDENCIAS
# ==============================================================================

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import json
from pathlib import Path
from datetime import datetime

# SDV
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.evaluation.single_table import evaluate_quality

# Configuración
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Paths
DATA_DIR = Path("../../v2/data/processed")
OUTPUT_DIR = Path(".")

# Cuántas filas sintéticas generar
N_SYNTHETIC = 1000

print("✅ Dependencias cargadas")
print(f"   SDV GaussianCopula")
print(f"   Random State: {RANDOM_STATE}")
print(f"   N Sintético: {N_SYNTHETIC}")

✅ Dependencias cargadas
   SDV GaussianCopula
   Random State: 42
   N Sintético: 1000


---
## 1. Carga de Datos de Entrenamiento

In [2]:
# ==============================================================================
# CARGA DE DATOS (SOLO TRAIN - CERO LEAKAGE)
# ==============================================================================

train_df = pd.read_parquet(DATA_DIR / "train_final.parquet")

print(f"📊 Datos cargados (SOLO TRAIN):")
print(f"   Shape: {train_df.shape}")
print(f"   Columnas: {len(train_df.columns)}")

# Verificar columnas zero-variance a excluir
zero_var_cols = []
for col in train_df.columns:
    if train_df[col].nunique() <= 1:
        zero_var_cols.append(col)
        
print(f"\n⚠️ Columnas zero-variance detectadas: {zero_var_cols}")
print(f"   (Serán excluidas del sintetizador)")

# Crear dataframe limpio para síntesis
train_clean = train_df.drop(columns=zero_var_cols, errors='ignore')
print(f"\n📋 Dataset para síntesis: {train_clean.shape}")

📊 Datos cargados (SOLO TRAIN):
   Shape: (296, 63)
   Columnas: 63

⚠️ Columnas zero-variance detectadas: ['tech_python', 'tech_big_data']
   (Serán excluidas del sintetizador)

📋 Dataset para síntesis: (296, 61)


---
## 2. Definición de Metadatos SDV

In [3]:
# ==============================================================================
# DEFINICIÓN DE METADATOS
# ==============================================================================

print("📝 Definiendo metadatos para SDV...")

# Crear metadata automáticamente
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train_clean)

# Mostrar tipos detectados
print(f"\n📋 Tipos detectados:")
for col, info in metadata.columns.items():
    sdtype = info.get('sdtype', 'unknown')
    print(f"   {col}: {sdtype}")
    
# Verificar metadata
metadata.validate()
print(f"\n✅ Metadatos validados correctamente")

📝 Definiendo metadatos para SDV...

📋 Tipos detectados:
   edad: numerical
   genero_m: categorical
   hab_1: numerical
   hab_2: numerical
   hab_3: numerical
   hab_4: numerical
   hab_5: numerical
   hab_6: numerical
   hab_7: numerical
   tech_programacion: categorical
   tech_java: categorical
   tech_desarrollo_web: categorical
   tech_desarrollo_movil: categorical
   tech_base_datos: categorical
   tech_machine_learning: categorical
   tech_inteligencia_artificial: categorical
   tech_analisis_datos: categorical
   tech_redes: categorical
   tech_telecomunicaciones: categorical
   tech_redes_inalambricas: categorical
   tech_sistemas_celulares: categorical
   tech_ciberseguridad: categorical
   tech_cloud_computing: categorical
   tech_sistemas_operativos: categorical
   tech_iot: categorical
   tech_automatizacion: categorical
   tech_robotica: categorical
   tech_electronica: categorical
   tech_instrumentacion: categorical
   tech_control: categorical
   tech_electricidad: ca

In [4]:
# ==============================================================================
# AJUSTE MANUAL DE METADATOS (si es necesario)
# ==============================================================================

# Asegurar que 'event' sea categórico/booleano
if 'event' in metadata.columns:
    metadata.update_column('event', sdtype='categorical')
    print("✏️ 'event' ajustado a categorical")

# Asegurar que 'duration' sea numérico
if 'duration' in metadata.columns:
    metadata.update_column('duration', sdtype='numerical')
    print("✏️ 'duration' ajustado a numerical")

# Asegurar que binarios tech_* sean categóricos
binary_cols = [c for c in train_clean.columns if c.startswith('tech_') or c == 'genero_m']
for col in binary_cols:
    if col in metadata.columns:
        metadata.update_column(col, sdtype='categorical')

print(f"✏️ {len(binary_cols)} columnas binarias ajustadas a categorical")

# Validar de nuevo
metadata.validate()
print("\n✅ Metadatos actualizados y validados")

✏️ 'event' ajustado a categorical
✏️ 'duration' ajustado a numerical
✏️ 51 columnas binarias ajustadas a categorical

✅ Metadatos actualizados y validados


---
## 3. Entrenamiento del Sintetizador

In [5]:
# ==============================================================================
# CONFIGURAR Y ENTRENAR GAUSSIANCOPULASYNTHESIZER
# ==============================================================================

print("🔧 Configurando GaussianCopulaSynthesizer...")

synthesizer = GaussianCopulaSynthesizer(
    metadata,
    enforce_min_max_values=True,
    enforce_rounding=True,
    numerical_distributions={
        'duration': 'truncnorm',  # Truncated normal para evitar negativos
        'edad': 'truncnorm'
    },
    default_distribution='norm'
)

print("\n🏋️ Entrenando sintetizador...")
synthesizer.fit(train_clean)

print("\n✅ Sintetizador entrenado exitosamente")

🔧 Configurando GaussianCopulaSynthesizer...

🏋️ Entrenando sintetizador...



✅ Sintetizador entrenado exitosamente


---
## 4. Generación de Datos Sintéticos

In [6]:
# ==============================================================================
# GENERAR DATOS SINTÉTICOS
# ==============================================================================

print(f"🧬 Generando {N_SYNTHETIC} filas sintéticas...")

synthetic_df = synthesizer.sample(num_rows=N_SYNTHETIC)

print(f"\n📊 Datos sintéticos generados:")
print(f"   Shape: {synthetic_df.shape}")
print(f"\n   Primeras filas:")
print(synthetic_df.head())

🧬 Generando 1000 filas sintéticas...



📊 Datos sintéticos generados:
   Shape: (1000, 61)

   Primeras filas:
   edad  genero_m  hab_1  hab_2  hab_3  hab_4  hab_5  hab_6  hab_7  \
0    25         1   0.47   0.46   0.48   0.42   0.59   0.75   0.66   
1    21         0   0.95   0.78   1.00   1.00   1.00   1.00   0.97   
2    25         1   0.94   0.92   0.60   0.93   0.74   1.00   0.54   
3    23         1   0.87   0.56   0.87   0.76   0.63   0.52   0.92   
4    25         1   0.47   0.76   0.68   0.50   0.47   0.47   0.57   

   tech_programacion  ...  tech_agroindustria  tech_matematica_aplicada  \
0                  0  ...                   0                         0   
1                  0  ...                   0                         0   
2                  1  ...                   0                         1   
3                  0  ...                   0                         0   
4                  0  ...                   0                         0   

   tech_fisica_aplicada  tech_economia_finanzas  tech_ge

---
## 5. Post-Procesamiento

In [7]:
# ==============================================================================
# POST-PROCESAMIENTO: ASEGURAR RESTRICCIONES DE DOMINIO
# ==============================================================================

print("🔧 Aplicando post-procesamiento...")

# 1. Duration debe ser > 0
min_duration = train_clean['duration'].min()
if 'duration' in synthetic_df.columns:
    before = (synthetic_df['duration'] <= 0).sum()
    synthetic_df['duration'] = synthetic_df['duration'].clip(lower=min_duration)
    print(f"   ✅ duration: {before} valores corregidos (clip a {min_duration:.2f})")

# 2. Event debe ser 0 o 1
if 'event' in synthetic_df.columns:
    synthetic_df['event'] = synthetic_df['event'].round().astype(int).clip(0, 1)
    print(f"   ✅ event: convertido a binario {synthetic_df['event'].unique()}")

# 3. Edad debe estar en rango razonable
if 'edad' in synthetic_df.columns:
    min_edad = train_clean['edad'].min()
    max_edad = train_clean['edad'].max()
    synthetic_df['edad'] = synthetic_df['edad'].clip(min_edad, max_edad).round().astype(int)
    print(f"   ✅ edad: clip a [{min_edad}, {max_edad}]")

# 4. Columnas binarias tech_* deben ser 0 o 1
binary_cols = [c for c in synthetic_df.columns if c.startswith('tech_') or c == 'genero_m']
for col in binary_cols:
    synthetic_df[col] = synthetic_df[col].round().astype(int).clip(0, 1)
print(f"   ✅ {len(binary_cols)} columnas binarias convertidas a 0/1")

# 5. Habilidades hab_* deben estar en [0, 1]
hab_cols = [c for c in synthetic_df.columns if c.startswith('hab_')]
for col in hab_cols:
    # Redondear a valores válidos: 0, 0.25, 0.5, 0.75, 1.0
    synthetic_df[col] = (synthetic_df[col] * 4).round() / 4
    synthetic_df[col] = synthetic_df[col].clip(0, 1)
print(f"   ✅ {len(hab_cols)} columnas hab_* normalizadas a [0, 0.25, 0.5, 0.75, 1.0]")

print("\n✅ Post-procesamiento completado")

🔧 Aplicando post-procesamiento...
   ✅ duration: 0 valores corregidos (clip a 0.58)
   ✅ event: convertido a binario [0 1]
   ✅ edad: clip a [21, 40]
   ✅ 51 columnas binarias convertidas a 0/1
   ✅ 7 columnas hab_* normalizadas a [0, 0.25, 0.5, 0.75, 1.0]

✅ Post-procesamiento completado


---
## 6. Validación de Calidad

In [8]:
# ==============================================================================
# VALIDACIÓN DE CALIDAD SDV
# ==============================================================================

print("📊 Evaluando calidad de los datos sintéticos...")

try:
    quality_report = evaluate_quality(
        real_data=train_clean,
        synthetic_data=synthetic_df,
        metadata=metadata
    )
    
    overall_score = quality_report.get_score()
    print(f"\n🏆 Score de Calidad General: {overall_score:.4f}")
    
except Exception as e:
    print(f"⚠️ No se pudo calcular el score de calidad: {e}")
    overall_score = None

📊 Evaluando calidad de los datos sintéticos...
Generating report ...



|          | 0/61 [00:00<?, ?it/s]|

(1/2) Evaluating Column Shapes: |          | 0/61 [00:00<?, ?it/s]|

(1/2) Evaluating Column Shapes: |██████████| 61/61 [00:00<00:00, 2188.76it/s]|


Column Shapes Score: 96.68%



...

(2/2) Evaluating Column Pair Trends: |█████████▌| 1742/1830 [00:04<00:00, 448.16it/s]|

(2/2) Evaluating Column Pair Trends: |█████████▊| 1787/1830 [00:04<00:00, 437.97it/s]|

(2/2) Evaluating Column Pair Trends: |██████████| 1830/1830 [00:04<00:00, 431.16it/s]|


Column Pair Trends Score: 90.59%



Overall Score (Average): 93.63%


🏆 Score de Calidad General: 0.9363


In [9]:
# ==============================================================================
# COMPARACIÓN ESTADÍSTICA
# ==============================================================================

print("📊 Comparación estadística Real vs Sintético:")
print("\n" + "="*70)

# Comparar estadísticas clave
comparison_cols = ['duration', 'event', 'edad']

for col in comparison_cols:
    if col in train_clean.columns and col in synthetic_df.columns:
        real_mean = train_clean[col].mean()
        synth_mean = synthetic_df[col].mean()
        real_std = train_clean[col].std()
        synth_std = synthetic_df[col].std()
        
        print(f"\n{col}:")
        print(f"   Real:      μ={real_mean:.3f}, σ={real_std:.3f}")
        print(f"   Sintético: μ={synth_mean:.3f}, σ={synth_std:.3f}")
        print(f"   Δ mean:    {abs(real_mean - synth_mean):.3f}")

# Comparar tasa de eventos
if 'event' in train_clean.columns:
    real_event_rate = train_clean['event'].mean()
    synth_event_rate = synthetic_df['event'].mean()
    print(f"\nEvent Rate:")
    print(f"   Real:      {real_event_rate:.1%}")
    print(f"   Sintético: {synth_event_rate:.1%}")
    print(f"   Δ:         {abs(real_event_rate - synth_event_rate):.1%}")

📊 Comparación estadística Real vs Sintético:


duration:
   Real:      μ=15.431, σ=10.992
   Sintético: μ=15.508, σ=8.716
   Δ mean:    0.077

event:
   Real:      μ=0.456, σ=0.499
   Sintético: μ=0.451, σ=0.498
   Δ mean:    0.005

edad:
   Real:      μ=25.176, σ=2.884
   Sintético: μ=25.265, σ=2.967
   Δ mean:    0.089

Event Rate:
   Real:      45.6%
   Sintético: 45.1%
   Δ:         0.5%


---
## 7. Guardar Resultados

In [10]:
# ==============================================================================
# GUARDAR DATOS SINTÉTICOS Y MODELO
# ==============================================================================

print("💾 Guardando resultados...")

# 1. Guardar datos sintéticos
synthetic_df.to_parquet(OUTPUT_DIR / "synthetic_data_copula.parquet", index=False)
print(f"   ✅ synthetic_data_copula.parquet ({len(synthetic_df)} filas)")

# 2. Guardar modelo sintetizador
synthesizer.save(OUTPUT_DIR / "synthesizer_model.pkl")
print(f"   ✅ synthesizer_model.pkl")

# 3. Guardar metadatos
metadata.save_to_json(OUTPUT_DIR / "synthesizer_metadata.json")
print(f"   ✅ synthesizer_metadata.json")

# 4. Guardar reporte de generación
report = {
    "metadata": {
        "date": datetime.now().isoformat(),
        "method": "GaussianCopulaSynthesizer",
        "sdv_version": "1.32.0",
        "random_state": RANDOM_STATE
    },
    "input": {
        "n_real": len(train_clean),
        "n_features": len(train_clean.columns),
        "excluded_cols": zero_var_cols
    },
    "output": {
        "n_synthetic": len(synthetic_df),
        "quality_score": float(overall_score) if overall_score else None
    },
    "statistics_comparison": {
        "duration": {
            "real_mean": float(train_clean['duration'].mean()),
            "synth_mean": float(synthetic_df['duration'].mean()),
            "real_std": float(train_clean['duration'].std()),
            "synth_std": float(synthetic_df['duration'].std())
        },
        "event_rate": {
            "real": float(train_clean['event'].mean()),
            "synthetic": float(synthetic_df['event'].mean())
        }
    },
    "files_generated": [
        "synthetic_data_copula.parquet",
        "synthesizer_model.pkl",
        "synthesizer_metadata.json"
    ]
}

with open(OUTPUT_DIR / "generation_report.json", 'w') as f:
    json.dump(report, f, indent=2)
print(f"   ✅ generation_report.json")

print("\n" + "="*50)
print("🎉 GENERACIÓN SINTÉTICA COMPLETADA")
print("="*50)

💾 Guardando resultados...
   ✅ synthetic_data_copula.parquet (1000 filas)
   ✅ synthesizer_model.pkl
   ✅ synthesizer_metadata.json
   ✅ generation_report.json

🎉 GENERACIÓN SINTÉTICA COMPLETADA


---
## Resumen

### Archivos Generados

| Archivo | Descripción |
|---------|-------------|
| `synthetic_data_copula.parquet` | 1000 filas sintéticas |
| `synthesizer_model.pkl` | Modelo GaussianCopula entrenado |
| `synthesizer_metadata.json` | Metadatos SDV |
| `generation_report.json` | Reporte de generación |

### Siguiente Paso

**Prompt 5: Entrenamiento con Datos Aumentados** - Entrenar modelos con combinaciones de datos reales y sintéticos.

---